In [1]:
import os
os.chdir('../..')
HOME = os.environ['HOME']

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas
import re
import json

from utils.util_pandas import PandasDF, create_agent_from_row

np.random.seed(7)
torch.manual_seed(7)

In [3]:
order2bps = {
    'bpsk': 1,
    'qpsk': 2,
    '8psk': 3,
    '16qam': 4,
    '64qam': 6
}

In [4]:
# Load run results
HOME_DIR = './data'
PANDAS_DATAFRAME_FILENAME = HOME_DIR + '/df.xls'
RESULTS_DIR = HOME_DIR + '/results'
RESULTS_KEY = 'results_filename'
MODELS_DIR = HOME_DIR + '/saved_models'
MODELS_KEY = 'model_filename'

DF = PandasDF(HOME_DIR=HOME_DIR, RESULTS_DIR=RESULTS_DIR, MODELS_DIR=MODELS_DIR,
              PANDAS_DATAFRAME_FILENAME=PANDAS_DATAFRAME_FILENAME,
              RESULTS_KEY=RESULTS_KEY, MODELS_KEY=MODELS_KEY)

In [5]:
DF.df.keys()

Index(['std_explore', 'restrict_energy', 'inner_steps', 'outer_batch_size',
       'nshots', 'model', 'SNR_db', 'stepsize_meta', 'hidden_layers',
       'stepsize_inner', 'first_order', 'meta-alg', 'learner',
       'inner_batch_size', 'bits_per_symbol', 'ntasks', 'optimizer', 'epochs',
       'seed', 'model_filename', 'results_filename'],
      dtype='object')

In [6]:
protocol = "shared_preamble"
experiment = "QPSK_neural_vs_selfalien"
train_dir = HOME + '/tmp/echo/experiments/%s/%s/' % (protocol, experiment)
output_dir = train_dir + 'results/'
with open(train_dir + "jobs.json", "r") as f:
    jobs = json.load(f)
print("%i job(s) present" % len(jobs))

150 job(s) present


In [12]:
for j, job in enumerate(jobs):
    print("Job %i" % j)
#     print(job['__meta__'])
#     print(job)
    query = {'meta-alg': 'none', 'ntasks': 1}
    query['bps'] = order2bps[job['__meta__']['mod_order'].lower()]
    query['seed'] = job['__meta__']['torch_seed']
    query['epochs'] = job['num_iterations']
    query['inner_batch_size'] = job['batch_size']
    query['SNR_db'] = job['train_SNR_db']
    for a, agentstr in enumerate(['agent1', 'agent2']):
        agent = job[agentstr]
        for md in ['mod', 'demod']:
            if agent[md + '_model'] == 'classic':
                continue
            q = query.copy()
            q.update(agent[md + "_params"])
            q['model'] = agent[md + "_model"]
            q['model'] = q['model'][0].upper() + q['model'][1:]
            q['learner'] = md + 'ulator'
            response = DF.search_rows(q)
            if response is None or len(response) == 0:
                results = np.load(output_dir + "{}.npy".format(j), allow_pickle=True)
                final_results = results[-1]
                final_db_off = final_results["db_off"]
                if np.mean(final_db_off) < 1.0:
                    q['results_filename'] = DF.save_results(results)
                    q['model_filename'] = DF.save_model(torch.load(output_dir + "{}-{}{}.wgts".format(j, md, a + 1)))
                    DF.add_row(q)
                else:
                    print("dB-off {:0.1f} too high, skipping".format(np.mean(final_db_off)))
            else:
                print("Already saved results of this experiment")

Job 0
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 1
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 2
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 3
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 4
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 5
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results

Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 50
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 51
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 52
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 53
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 54
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 55
Already saved r

Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 100
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 101
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 102
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 103
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Job 104
Already saved results of this experiment
Already saved results of this experiment
Already saved results of this experiment
Already saved res

Already saved results of this experiment
Already saved results of this experiment
Job 149
dB-off 1.2 too high, skipping
dB-off 1.2 too high, skipping
dB-off 1.2 too high, skipping
dB-off 1.2 too high, skipping


In [13]:
DF.df

,stepsize_cross_entropy,loss_type,cross_entropy_weight,max_amplitude,stepsize_sigma,stepsize_mu,min_std,max_std,lambda_prob,lambda_p,...,meta-alg,learner,inner_batch_size,bits_per_symbol,ntasks,optimizer,epochs,seed,model_filename,results_filename
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_1.npy,./data/results/results_1.npy
2,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_2.npy,./data/results/results_2.npy
3,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_3.npy,./data/results/results_3.npy
4,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_4.npy,./data/results/results_4.npy
5,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_5.npy,./data/results/results_5.npy
6,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_6.npy,./data/results/results_6.npy
7,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_7.npy,./data/results/results_7.npy
8,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_8.npy,./data/results/results_8.npy
9,None,None,None,None,None,None,None,None,None,None,...,MAML,demodulator,256,6,16,sgd,1000,777,./data/saved_models/agent_9.npy,./data/results/results_9.npy
